In [ ]:
import pandas as pd
from os import path, environ, makedirs
import requests as re
from io import BytesIO
from dotenv import load_dotenv
from unidecode import unidecode
import numpy as np

In [ ]:
load_dotenv()

# Carregando os dados extraídos no notebook anterior

Neste notebook, vamos utilizar os dados extraídos e salvos pelo notebook `05 ObservaSampa - extração.ipynb`.

Primeiro, vamos carregar os dados de subprefeituras que serão utilizados no Qlik Sense.

In [ ]:
output_dir = path.join('dados', 'urbanismo')
if not path.exists(output_dir):
    makedirs(output_dir)

In [ ]:
dist_mun = pd.read_csv(path.join(output_dir, 'Distrito.csv'),
                        sep=';', decimal=',', encoding='utf8', dtype=str)

In [ ]:
dist_censo = pd.read_csv(path.join(output_dir, 'Setor-censitario.csv'),
                          sep=';', decimal=',', encoding='utf8', dtype=str)

In [ ]:
subs = pd.read_csv(path.join(output_dir, 'Subprefeituras.csv'),
                              sep=';', decimal=',', encoding='utf8', dtype=str)

## Tratamento do Indicador domicilios não ocupados (censo 2022)

In [ ]:
def construir_df_ido(dist_mun, dist_censo, subs):
    dist_mun = dist_mun.loc[:, ~dist_mun.columns.duplicated(keep='last')].copy()
    dist_censo = dist_censo[dist_censo["CD_MUN"].astype(str) == "3550308"].copy()
    dist_censo["nm_distrito_municipal"] = dist_censo["NM_DIST"].apply(unidecode).str.upper()
    dist_mun = dist_mun.merge(
        dist_censo[["nm_distrito_municipal", "v0003", "v0007"]],
        on="nm_distrito_municipal",
        how="left"
    ).rename(columns={"v0003": "total_domicilios_particulares", "v0007": "domicilios_particulares_ocupados"})
    for c in ["total_domicilios_particulares", "domicilios_particulares_ocupados"]:
        dist_mun[c] = pd.to_numeric(dist_mun[c], errors="coerce")
    dist_mun["domicilios_particulares_nao_ocupados"] = dist_mun["total_domicilios_particulares"] - dist_mun["domicilios_particulares_ocupados"]
    dist_mun["cd_subprefeitura"] = pd.to_numeric(dist_mun["cd_identificador_subprefeitura"], errors="coerce").astype("Int64").astype(str).str.zfill(2)
    subs = subs.copy()
    subs["cd_subprefeitura"] = pd.to_numeric(subs["cd_subprefeitura"], errors="coerce").astype("Int64").astype(str).str.zfill(2)
    dom_subs = dist_mun.groupby("cd_subprefeitura", as_index=False).agg({
        "total_domicilios_particulares": "sum",
        "domicilios_particulares_ocupados": "sum",
        "domicilios_particulares_nao_ocupados": "sum"
    })
    df_ido = subs.merge(dom_subs, on="cd_subprefeitura", how="left")[[
        "cd_subprefeitura",
        "nm_subprefeitura",
        "domicilios_particulares_nao_ocupados",
        "domicilios_particulares_ocupados",
        "total_domicilios_particulares"
    ]].copy()
    df_ido["nm_subprefeitura"] = df_ido["nm_subprefeitura"].astype(str).str.strip()
    df_ido["tx_nao_ocupados_pct"] = np.where(
        pd.to_numeric(df_ido["total_domicilios_particulares"], errors="coerce").gt(0),
        df_ido["domicilios_particulares_nao_ocupados"] / df_ido["total_domicilios_particulares"] * 100,
        np.nan
    )
    return df_ido.sort_values("cd_subprefeitura").reset_index(drop=True)

In [ ]:
df_ido = construir_df_ido(dist_mun, dist_censo, subs)

In [ ]:
_dest = path.join("data_output", "urbanismo")
if not path.exists(_dest):
    makedirs(_dest)

In [ ]:
df_ido.to_csv(
    path.join(_dest, "Indicador domicilios não ocupados (censo 2022).csv"),
    index=False,
    sep=";",
    decimal=",",
    encoding="latin1"
)

In [ ]:
df_ido

# Armazenamento

Finalmente, vamos exportar os dados em formato csv compatível com o Qlik e no padrão do excel para português do Brasil.

In [ ]:
base_path = path.join('data_output', 'urbanismo')

if not path.exists(base_path):
    makedirs(base_path)

for name, df in [
                 ('indicador_domicilios_nao_ocupados', df_ido),
                 ]:
    filepath = path.join(base_path, f'{name}.csv')

    df.to_csv(filepath,
              index=False,
              sep=';',
              decimal=',',
              encoding='latin1')